In [1]:
import bs4
import pandas as pd
import pickle
import random
import requests
from time import sleep
from tqdm import tqdm 

In [2]:
main_url = 'https://vseotzyvy.ru/' 
req = requests.get(main_url)

In [3]:
parser = bs4.BeautifulSoup(req.text, 'lxml')
all_categories = parser.findAll('div', 
                                attrs={'class':'home_sub_cat clearfix'})

Here we get links for all categories.

In [4]:
hrefs = set()
for columns in all_categories:
    cur_categories = columns.select('a')
    for num, href in enumerate(cur_categories):
        if num != len(cur_categories) - 1:
            hrefs.add(href.get('href'))

In [5]:
for href in hrefs:
    req = requests.get(main_url + href)
    

And here we get links to all objects in all categories.

In [6]:
items_set = set()
for href in tqdm(hrefs):
    item_page_prev = []
    for i in range(1, 100):
        sleep(0.3)
        params = {'page': i}
        req = requests.get(main_url + href, params)
        parser = bs4.BeautifulSoup(req.text, 'lxml')
        item_page = parser.findAll('h3')
        if len(item_page) == 0:  # no more pages available
            break
        if item_page == item_page_prev:  # if prev_page repeats then break
            break
        for item in item_page:
            items_set.add(item.select('a')[0].get('href'))
        item_page_prev = item_page

100%|██████████| 122/122 [10:22<00:00,  5.11s/it]


In [7]:
with open('items_set.pickle', 'rb') as handle:
    items_set = pickle.load(handle)  # Save it just in case

In [8]:
def binarize_mark(mark):
    if mark < 4:
        return 0
    return 1

def get_train_df(items_num, pages_max, emotion):
    marks = []
    reviews = []
    for item_url in tqdm(random.sample(items_set, items_num)):
        sleep(0.5)
        for page in range(1, pages_max + 1):
            if pages_max > 1:
                sleep(0.3)
            '''To avoid the problem of unbalanced classes, we will leave
               reviews of only one sentiment and collect them separately'''
            params = {'page': page, 'sort': emotion}  
            req = requests.get(main_url + item_url, params)
            parser = bs4.BeautifulSoup(req.text, 'lxml')
            marks_html = parser.findAll('span',  attrs={'class': 'bold', 'itemprop': 'ratingValue'})
            reviews_html = parser.findAll('span',  attrs={'itemprop': 'reviewBody'})
            if (len(marks_html) == 0) or (len(marks_html) != len(reviews_html)):  # no more pages available
                break
            for mark in marks_html:
                marks.append(int(mark.text))
            for review in reviews_html:
                reviews.append(review.text)
        # print(len(marks), len(reviews))

    df = pd.DataFrame({'review': reviews, 'mark': marks})
    df['mark'] = df['mark'].apply(binarize_mark)
    return df

In [10]:
neg_df = get_train_df(2000, 1, 'negative')
pos_df = get_train_df(500, 1, 'positive')  # There are much fewer negative ones, so we will parse more pages on them
train_df = pd.concat((neg_df, pos_df))

train_df.to_csv('sentiment_df.csv', index=False)

100%|██████████| 500/500 [06:34<00:00,  1.27it/s]
